# SBS SMK Test

This notebook should be used as a test for ensuring correct SBS image loading and processing before running a snakemake workflow.
Cells marked with `SET PARAMETERS` contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

In [ ]:
import os
import shutil

original_tifs_dir = "/lab/barcheese01/screens/denali/input_sbs_tif"
new_tifs_dir = "/lab/barcheese01/rkern/OpticalPooledScreens/example_analysis/1_sbs/input/sbs_tifs"
cycles = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
wells = ["A1", "A2"]
tiles = [1, 100]

PREPROCESS_PATTERN = "10X_c{cycle}-SBS-{cycle}_{well}_Tile-{tile}.sbs.tif"

# Generate all possible filenames based on the pattern
valid_filenames = {
    PREPROCESS_PATTERN.format(cycle=cycle, well=well, tile=tile)
    for cycle in cycles
    for well in wells
    for tile in tiles
}

# Create new directory if it doesn't exist
os.makedirs(new_tifs_dir, exist_ok=True)

# Iterate and copy only files that match the generated patterns
for filename in os.listdir(original_tifs_dir):
    if filename in valid_filenames:
        source_path = os.path.join(original_tifs_dir, filename)
        destination_path = os.path.join(new_tifs_dir, filename)
        shutil.copy(source_path, destination_path)
        print(f"Copied: {source_path} to {destination_path}")


In [ ]:
import os
import shutil

tifs_dir = "/lab/barcheese01/screens/denali/illumination_correction"
new_tifs_dir = "/lab/barcheese01/rkern/OpticalPooledScreens/example_analysis/1_sbs/input/sbs_ic_tifs"
wells = ["A1", "A2"]
# Define segmentation cycle
SEGMENTATION_CYCLE = 11

# Create new directory if it doesn't exist
os.makedirs(new_tifs_dir, exist_ok=True)

# Iterate through files in the directory and copy matching files
for filename in os.listdir(tifs_dir):
    for well in wells:
        # Construct the expected filename pattern for each well
        icf_file_pattern = f"10X_c{SEGMENTATION_CYCLE}-SBS-{SEGMENTATION_CYCLE}_{well}.sbs.illumination_correction.tif"
        
        # Copy the file if it matches the pattern
        if filename == icf_file_pattern:
            source_path = os.path.join(tifs_dir, filename)
            destination_path = os.path.join(new_tifs_dir, filename)
            shutil.copy(source_path, destination_path)
            print(f"Copied: {source_path} to {destination_path}")


## Imports

In [ ]:
from IPython.display import display
from microfilm import microplot
from microfilm.microplot import Micropanel
import matplotlib.pyplot as plt
import matplotlib

from ops.sbs_smk import Snake_sbs
import ops
from ops.imports_ipython import *
from ops.qc import *

## SET PARAMETERS

The following cell contains crucial variables that need to be set according to your specific experimental setup and data organization. Please review and modify these variables as needed before proceeding with the analysis.

### Loading, aligning, detecting candiate reads

- `SBS_CYCLES`: The number and order of SBS cycles
- `WELLS` and `TILES`: The well and tile you want to analyze (for this notebook, we're using one well/tile combination for testing)
- `CHANNELS` and `preprocess_pattern`: The channel names and file naming conventions
- `DISPLAY_RANGES` and `LUTS`: The display ranges and color schemes (Look-Up Tables) for your image data

In [ ]:
# Output directory for notebook results
INPUT_FILES_DIR = "input"
OUTPUT_FILES_DIR = "output"